In [ ]:
#TASK LIST 22/09/2020
#get forward and backward pass to work on the dataset DONE 
#then montecarlo simulations with critical path DONE FOR 1X
#monte carlo critical path - DONE BUT SLOW 
#use real data set -- make sure works with critical NOW 24/09/2020
# add deliverables to have more structure WP1 ETC
# edit regex ids for longer than 26 letters 


In [2]:
import re 
import numpy as np 
import os 
import pandas as pd
import matplotlib.pyplot as plt 
from scipy import stats
from datetime import datetime, timedelta, date

NameError: name 'df' is not defined

In [4]:

#working on adding the real data into the simulation 

#b = np.r_[0:29, 34:38]

    


NameError: name 'import_data' is not defined

In [5]:
class Task(object):
    def __init__(self, description, ID, predecessors, days1):
        self.description = description 
        self.ID = ID
        self.predecessors = predecessors
        self.days1 = days1
        self.earlyStart = 0 
        self.earliestFinish = 0
        self.sucessors = []
        self.latestStart = 0
        self.latestFinish = 0
        self.slack = 0
        self.critical = ' '
    #compute the slack for a task 
    def compute_slack(self):
        self.slack=self.latestFinish-self.earliestFinish 
        if self.slack > 0:
            self.critical = 'NO'
        else: 
            self.critical = 'YES'



In [33]:
def import_data(csvfile):
    dataframe=pd.read_csv(csvfile)
    return dataframe

def time_nan(dataframe): #converts panda dataframe into more useful format for datetime manipulation
    F = len(dataframe['START'])
    p = 0 
    while p < F: 
    
        if pd.isna(dataframe['START'][p]):
            dataframe['START'][p] = date.today()
        else:
            dataframe['START'][p]==dataframe['START'][p]
        p+=1 
    return dataframe 

def data_time(dataframe):       # converts dates to datetime objects and finds length of tasks in days
    dataframe['START']=pd.to_datetime(dataframe['START'], dayfirst=True)
    dataframe['END']=pd.to_datetime(dataframe['END'], dayfirst=True)

    dataframe['timediff predicted'] = dataframe['END']-dataframe['START']
    dataframe['timediff predicted'] = pd.to_numeric(dataframe['timediff predicted'].dt.days) 
    return dataframe
#function to compute the number of days the task will be carried out for 
#may have to convert to timefile .. not so simple with different start and end dates
def calculate_days(dataframe):
    
    dataframe['START']=pd.to_datetime(dataframe['START'])
    dataframe['END']=pd.to_datetime(dataframe['END'])
    dataframe['timediff'] = dataframe['END']-dataframe['START']
    dataframe['DAYS']=pd.to_numeric(dataframe['timediff'].dt.days)
    #dataframe['days']=(dataframe['enddate']-dataframe['startdate']).days 
    return dataframe 


In [7]:
#import the csv file and add in the columns. maybe do this on excel. 
def create_tasks(dataframe):
    task_object = []
    for i in range(len(dataframe)):
        task_object.append(Task(dataframe['description'][i], dataframe['ID'][i], dataframe['predecessors'][i], dataframe['days1'][i]))
    return (task_object)
    
def forward_pass(task_object):
    #rule 1 the earliest start of activity with no predecessors = 0 
    #rule 2 the earliest start is max early finish of predecessors 
    #rule 3 early finish = sum of ES and activity time 
    for task in task_object:
        
        if type(task.predecessors) is str:
            task.predecessors=task.predecessors.upper()
            ef = []
            for j in task.predecessors :
                for t in task_object:
                    if t.ID == j :
                        ef.append(t.earliestFinish)
                task.earlyStart = max(ef)
            del ef 
        else: 
            task.earlyStart = 0 
        task.earliestFinish = task.earlyStart + task.days1
        

In [8]:
def backward_pass(task_object):
    pred=[]
    eF=[]
    #forward pass is carried out first 
    #gather earliest finish of all tasks and fill predecessors 
    #compute the successors of each task 
    for task in task_object:
        
        if type(task.predecessors)==str:
            
            for j in task.predecessors:
                pattern=re.compile(r' [A-Z]') #using regex expressions. limitation of 26 tasks need to convert to numbers
                match=pattern.finditer(j)
                for r in match:
                    pred.append(j) #fill predecessor list of all tasks 
                    #fill successors 
                    for m in task_object:
                        if m.ID == j:
                            m.successors.append(task.ID)   
        eF.append(task.earliestFinish)
        
    for task in reversed(task_object):
        if task.ID not in pred:
            task.latestFinish=max(eF)
            
        else:
            minLS = []
            for x in task.successors:
                for t in (task_object):
                    if t.ID == x:
                        minLS.append(t.latestStart)
            task.latestFinish = min(minLS) 
            del minLS
        task.latestStart = task.latestFinish - task.days1


In [9]:
#this simulation uses the startdate and finds number of days predicted based on a uniform distribution of random numbers

        
def randomise_days(dataframe):
    dataframe['days1'] = np.nan
    timediff=dataframe['days']
    


    N = len(dataframe)
    ran = np.zeros(N, dtype=float)


    for p in range(N):
        ran[p] = np.random.uniform(low=0.0, high=1.0, size=None)
        dataframe['days1'][p]=timediff[p]-0.1*timediff[p]+timediff[p]*0.2*ran[p]
        dataframe['days1'][p]=round(dataframe['days1'][p], 0)
    return dataframe 

In [10]:
def slack(task_object):
    for task in task_object:
        task.compute_slack() #dont understand this as well
        
        
def update_data_frame(df, task_object):
    df2 = pd.DataFrame({'description' : df['description'], 'ID' : df['ID'], 'days1':df['days1'], 
                        'ES':pd.Series([task.earlyStart for task in task_object]), 
                        'EF':pd.Series([task.earliestFinish for task in task_object]), 
                        'LS':pd.Series([task.latestStart for task in task_object]),
                        'LF':pd.Series([task.latestFinish for task in task_object]),
                        'CRITICAL':pd.Series([task.critical for task in task_object])})
    return df2

                 
         
    

df1=importData('CRITICALPATHdates.csv')

df1



In [11]:
def main_critical():
    os.system('clear')
    df1=import_data('CRITICALPATHdates.csv')[0:24] #for alphabet reasons  
    df1=calculate_days(df1)
    task_object = create_tasks(df1)
    forward_pass(task_object)
    backward_pass(task_object)
    slack(task_object)
    finaldf=update_data_frame(df1, task_object)
    print(df1)
    print(finaldf)
    


In [12]:
def main_monte_critical():
    os.system('clear')
    df1=import_data('CRITICALPATHdates.csv')[0:24] #for alphabet reasons  
    df1=calculate_days(df1)

    T = 100  #loop through and append the latest finish at the end 
    latestfinishes = np.zeros(T, dtype='O')
    for i in range(T):
        df2 = randomise_days(df1)
        task_object = create_tasks(df2)
        forward_pass(task_object)
        backward_pass(task_object)

        #slack(task_object)
        df2 = update_data_frame(df2, task_object)
        latestfinishes[i] = df2['LF'][1]

        #finaldf=update_data_frame(df1,task_object)
    plt.hist(latestfinishes, bins = 7)
    plt.show() #do subplots work..
    plt.hist(latestfinishes, bins=20)
    plt.show()

In [38]:
df = import_data('collab-csv-convert-.csv') #best thing to do is concactenate the two tables

df

#df4  = time_nan(df)
#bool_series = df['END'].notnull()
#df4 = df[bool_series] #need a column in case the end date is before the start date ... 
#changing the index to match the new dataframe or the loops won't work.. when getting rid of nan
#df4 = data_time(df)

#df4



,Unnamed: 0,LABEL,COUNTRY,INSTITUTION,DESCRIPTION,CHFK,CHFK.1,START,END
0,R&D,KE2736,France,CEA,Thermal Design of Superconducting High Field M...,180.0,180.0,1/1/2016,12/31/2018
1,NaN,KE3298,UK,Manchester,Beam instrumentation,1070.0,317.0,4/1/2016,12/31/2020
2,NaN,KE3299,UK,Manchester,Cold powering: DFBX for String,542.0,26.0,1/1/2017,12/31/2020
3,NaN,KN4526,UK,ASTeC+Dundee,Laser treatment prototype (LESS),150.0,0.0,NaN,3/31/2021
4,NaN,KE4417,Italy,INFN,High-order corrector magnets + prototypes,4450.0,1045.0,3/1/2020,1/1/2023
5,NaN,KE3797,Spain,CIEMAT,Nested orbit correctors + prototypes,6200.0,3040.0,4/1/2019,3/1/2023
6,NaN,KN4154,China,IHEP,D2 Correctors,2600.0,2225.0,1/1/2018,1/1/2022
7,NaN,KE3082,Sweden,Uppsala Univ.,Cold testing of corrector magnets and crab ca...,580.0,30.0,1/1/2016,12/31/2020
8,NaN,KR4454,Japan,KEK,D1 magnet model and cold mass,8850.0,8850.0,NaN,3/31/2022
9,NaN,KE3084/3083,Italy,INFN,D2 model + prototype+ Magnet,9000.0,2900.0,7/1/2019,12/31/2019
